# Lesson 1: Simple ReAct Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [11]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [12]:
client = OpenAI()

In [13]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [14]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [15]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [16]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [17]:
def calculate(what):
    try:
        return eval(what)
    except Exception as e:
        return str(e)

def get_country_capital(country):
    capitals = {
        "France": "Paris",
        "India": "New Delhi",
        "Japan": "Tokyo",
        "USA": "Washington, D.C."
    }
    return f"The capital of {country} is {capitals.get(country, 'unknown')}."
    
known_actions = {
    "calculate": calculate,
    "get_country_capital": get_country_capital
}


In [18]:
abot = Agent(prompt)

In [19]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [20]:
result = average_dog_weight("Toy Poodle")

In [21]:
result

'a toy poodles average weight is 7 lbs'

In [22]:
next_prompt = "Observation: {}".format(result)

In [23]:
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [24]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [25]:
abot = Agent(prompt)

In [26]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [27]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [28]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [29]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [30]:
abot(next_prompt)

'Thought: Now that I have the average weights of both dogs, I can calculate their combined weight by adding the two values together.\nAction: calculate: 37 + 20\nPAUSE'

In [31]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [32]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Add loop 

In [33]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [34]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [36]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE


Exception: Unknown action: average_dog_weight: Border Collie

In [37]:

# Example query: Should output reasoning and result using tools
query("What is the capital of France plus the length of its name?")


Thought: The capital of France is Paris. I need to calculate the length of the name "Paris" and then add it to the capital's name.
Action: calculate: 5 + len("Paris")
PAUSE
 -- running calculate 5 + len("Paris")
Observation: 10
Answer: The capital of France plus the length of its name is 10.


In [38]:
def get_capital(country):
    capitals = {
        "France": "Paris",
        "India": "New Delhi",
        "USA": "Washington, D.C.",
        "Germany": "Berlin"
    }
    return capitals.get(country, "Unknown")

def string_length(text):
    return len(text)


In [39]:
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,  # optional
    "get_capital": get_capital,
    "string_length": string_length
}


In [40]:
prompt = """
You are a smart agent that thinks step-by-step to answer questions. 
You can use the following tools:

- calculate: evaluate safe arithmetic expressions (e.g., 3 + 2)
- get_capital: find the capital of a country (e.g., get_capital: France)
- string_length: find the number of characters in a string (e.g., string_length: Paris)

Always format actions as:
Action: tool_name: input_text

Wait for the Observation before continuing. Then answer with:
Answer: your final answer here.
"""


In [41]:
query("What is the capital of France and how long is its name?")


Action: get_capital: France
 -- running get_capital France
Observation: Paris
Action: string_length: Paris
 -- running string_length Paris
Observation: 5
Answer: The capital of France is Paris, and its name is 5 characters long.
